In [1]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
URL='https://arxiv.org/list/cs.AI/recent'


In [4]:
response = requests.get(URL)

In [5]:
html_content=response.text
#html_content

In [6]:
soup = BeautifulSoup(html_content, 'html.parser')


In [7]:
articles_section = soup.find('dl', id='articles')


In [8]:
# Extract PDF links
if articles_section:
    pdf_links = []
    for entry in articles_section.find_all('a', href=True):
        if entry['href'].startswith('/pdf/'):
            pdf_link = f"https://arxiv.org{entry['href']}"  # Construct full link
            pdf_links.append(pdf_link)

In [9]:
pdf_links

['https://arxiv.org/pdf/2410.12784',
 'https://arxiv.org/pdf/2410.12631',
 'https://arxiv.org/pdf/2410.12539',
 'https://arxiv.org/pdf/2410.12509',
 'https://arxiv.org/pdf/2410.12409',
 'https://arxiv.org/pdf/2410.12389',
 'https://arxiv.org/pdf/2410.12376',
 'https://arxiv.org/pdf/2410.12375',
 'https://arxiv.org/pdf/2410.12361',
 'https://arxiv.org/pdf/2410.12288',
 'https://arxiv.org/pdf/2410.12219',
 'https://arxiv.org/pdf/2410.12207',
 'https://arxiv.org/pdf/2410.12126',
 'https://arxiv.org/pdf/2410.12112',
 'https://arxiv.org/pdf/2410.12031',
 'https://arxiv.org/pdf/2410.11905',
 'https://arxiv.org/pdf/2410.11900',
 'https://arxiv.org/pdf/2410.12774',
 'https://arxiv.org/pdf/2410.12773',
 'https://arxiv.org/pdf/2410.12772',
 'https://arxiv.org/pdf/2410.12771',
 'https://arxiv.org/pdf/2410.12761',
 'https://arxiv.org/pdf/2410.12759',
 'https://arxiv.org/pdf/2410.12730',
 'https://arxiv.org/pdf/2410.12728',
 'https://arxiv.org/pdf/2410.12720',
 'https://arxiv.org/pdf/2410.12707',
 

In [10]:
soup = BeautifulSoup(html_content, 'html.parser')

In [11]:
papers = []
for entry in soup.find_all('div', class_='list-title'):
        title = entry.text.strip().replace("Title:", "").strip()
        #link = entry.find_previous_sibling('div', class_='list-identifier').a['href']
        #link = f"https://arxiv.org{link}.pdf"  # Direct PDF link
        papers.append(title)

In [13]:
papers_with_links = list(zip(papers, pdf_links))

In [21]:
pdf_paths = []


In [22]:
for title, pdf_link in papers_with_links:
    response = requests.get(pdf_link)

    # Save the PDF with the title as the filename
    filename = f"{title}.pdf"
    with open(filename, 'wb') as f:
        f.write(response.content)
        pdf_paths.append(filename)

In [16]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 72.9 MB/s eta 0:00:00


In [17]:
import pdfplumber

In [18]:
text=""

In [23]:
for pdf_path in pdf_paths:
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()

**Till here, our web crawler is prepared, it has scraped the webpage provided, stored the links in a list , downloaded all the pdfs through that download links. Also, now it has extracted the text from the pdfs and storerd it in a list named as text.**

Now we will start with text preprocessing, this part involves the cleaning of the dataset, here we will be applying various techniques like lemitization, stemming, stopwords removal, Part of speech tagging and tokenization

In [25]:

from nltk.stem import WordNetLemmatizer

In [26]:
lemmatizer=WordNetLemmatizer()

In [27]:
from nltk.corpus import stopwords

In [29]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [30]:
stopword = stopwords.words('english')

In [32]:
stopword # these are the stopwords which we are going to remove from our dataset to refine it

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [34]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [35]:
sentences = nltk.sent_tokenize(text)

In [37]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [38]:
for i in range (len(sentences)):
  words=nltk.word_tokenize(sentences[i])
  for j in range (len(words)):
    if words[j] not in stopword:
      words[j]=lemmatizer.lemmatize(words[j])
  sentences[i]=' '.join(words)

Till now our dataset of text is cleaned by removing stopwords and lemmatizing the remaing words. Futher from here i have to encode my texts after which finally i can use it to do fine tuning of GPT or BERT on dataset which i have prepared.